# Atlas Search

#### dependency

In [41]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from pymongo.operations import SearchIndexModel

#### string constant, variable defenition

In [42]:
MONGODB_DATABASE_NAME = "cg"
MONGODB_COLLECTION_NAME = "bkg"

SEARCH_INDEX = 'bkg_search_index'

### Mongo DB

In [43]:
load_dotenv()
MONGODB_CONNECTION_STRING = os.environ["MONGODB_URI"]

mongo_client = MongoClient(MONGODB_CONNECTION_STRING)
mongo_db = mongo_client[MONGODB_DATABASE_NAME]
bkg_collection = mongo_db[MONGODB_COLLECTION_NAME]

##### 데이터 예시
```python
{
    "_id": {"$oid":"66e382fcae38f2b322d89a39"},
    "email_path": "  databooking bookingdatacherry20240715702 5 pdf",
    "booking close time": "2024-06-26 17:00",
    "booking request summary": "Book 45 High Cubic Reefer Container x 9 from DUBAI to HAMBURG",
    "cargo close time": "2024-06-28 17:00",
    "commodity items": "FRESH JUJUBES",
    "container size": "45 HIGH CUBIC CONTAINER",
    "container type": "REEFER CONTAINER",
    "container unit": "9",
    "customer name": "A_SAQR_INTERNATIONAL_FREIGHT",
    "document close time": "2024-06-27 17:00",
    "estimated arrival time (discharging port)": "2024-07-19 10:00",
    "estimated arrival time (loading port)": "2024-06-30 17:00",
    "estimated departure time (loading port)": "2024-07-01 20:00",
    "from": "vijay@asaqrinternational.com",
    "hs code": "0810.90",
    "place of discharging": "HAMBURG",
    "place of loading": "DUBAI",
    "remark": "REEFER CONTAINER, VALUE: USD 30,000, TEMPERATURE: +8°C",
    "salutation": "Dear, Sir",
    "shipper name": "CRESCENT MOON TRADING CO.",
    "subject": "mv.ARONIA V.202407W_Booking for HAMBURG_45 Reefer Container x 9",
    "to": "pic@cherry.com",
    "vessel name": "ARONIA",
    "voyage number": "202407W"
}
```

### Create Index

In [111]:
##### Execute this code ONLY ONCE initially. #####

# Create dynamic search index if it doesn't exist
bkg_search_indices = bkg_collection.list_search_indexes().to_list()
bkg_search_indices_names = [dic['name'] for dic in bkg_search_indices]
print("\n<Search Index>")
for dic in bkg_search_indices:
    print(f"{dic['name']} ::: 'definition': {dic['latestDefinition']}")

if SEARCH_INDEX not in bkg_search_indices_names:
    print(f"Index '{SEARCH_INDEX}' not found. Creating index ...")

    # Search Index 정의
    search_index_model = SearchIndexModel(
        definition={
            'mappings': {                           # 인덱싱 대상 필드 지정
                'dynamic': True,                    # 동적 맵핑 - 모든 필드를 대상으로 지정
                'fields': {                         # 정적 맵핑할 필드 지정
                    'email_path': [                 # 'email_path' 필드
                        {
                            'type': "string",                           # 데이터 타입
                            'analyzer': "email_path_analyzer",          # 인덱싱에 사용할 analyzer 지정 : Atlas Search 내장 analyzer or custom analyzer
                            'searchAnalyzer': "email_path_analyzer"     # 검색 input을 분석할 analyzer 지정 : Atlas Search 내장 analyzer or custom analyzer
                        }
                    ]
                }
            },
            'analyzer': "custom_analyzer",          # 인덱싱에 사용할 analyzer 지정 : Atlas Search 내장 analyzer or custom analyzer
            'searchAnalyzer': "custom_analyzer",    # 검색 input을 분석할 analyzer 지정 : Atlas Search 내장 analyzer or custom analyzer
            'analyzers': [                          # 이 인덱스에서 사용할 custom analyzer 정의 - 여러 개 선언 가능
                {
                    'name': "custom_analyzer",
                    'tokenizer': {                  # 텍스트 데이터를 개별 청크로 분할하는 방식 설정
                        'type': "regexSplit",       # tokenizer 유형 - 정규표현식 기반의 seperator를 사용하여 청크 분할
                        'pattern': "[_. ]+"          # 'regexSplit' 타입 추가 옵션 : 정규표현식 패턴 지정
                    },
                    'tokenFilters': [                       # 토큰 분할 후 필터링 작업 설정
                        {
                            'type': "lowercase"             # 대문자 -> 소문자 변환
                        },
                        {
                            'type': "englishPossessive"     # 단어 끝의 소유격('s) 제거
                        },
                        {
                            'type': "length",               # 토큰 길이 제한 : 최소/최대 길이 지정
                            'min': 2                        # 'length'타입 추가 옵션 : 최소 길이 제한 - 1글자짜리 토큰은 fuzzy 검색에서 변수가 너무 많다
                        },
                        {
                            'type': "stopword",             # 지정된 단어에 해당하는 토큰 제거
                            'tokens': [ "booking", "cargo", "commodity", "container", "customer", "document", "place", "remark", "shipper", "vessel", "voyage" ],
                            'ignoreCase': True              # 'stopword' 타입 추가 옵션 : 대소문자 구분 무시 여부 - True로 지정하면 "cargo" == "Cargo"로 인식함
                        }
                    ]
                },
                {
                    'name': "email_path_analyzer",
                    'charFilters': [                # 데이터 검사 및 필터링 작업 설정
                        {
                            'type': "mapping",      # 지정된 문자를 다른 문자로 대체하는 방식
                            'mappings': {           # 'mapping' 타입 추가 옵션
                                "_": ""             # underscore "_"를 공백문자 ""로 대체
                            }
                        }
                    ],
                    'tokenizer': {
                        'type': "regexSplit",
                        'pattern': "[/.]+"
                    },
                    'tokenFilters': [
                        {
                            'type': "length",
                            'min': 10
                        },
                        {
                            'type': "lowercase"
                        }
                    ]
                }
            ]
        },
        name=SEARCH_INDEX,      # Search Index명 지정
        type="search"           # Index 유형 지정 : "search" -> search index / "vectorSearch" -> vector search index
    )

    # Search Index 생성
    bkg_collection.create_search_index(model=search_index_model)
    
    print("Index creation finished.")


<Search Index>
Index 'bkg_search_index' not found. Creating index ...
Index creation finished.


### run search

In [142]:
# QUERY = ""
# QUERY = "booking_data_cherry20240715702 5"
# QUERY = "booking_data_cherry20240715702"
# QUERY = "A_SAQR_INTERNATIONAL_FREIGHT"
# QUERY = "A SAQR INTERNATIONAL FREIGHT"
# QUERY = "A_SAQR"
# QUERY = "A SAQR"
# QUERY = "AL WASL"
# QUERY = "SAQR"
# QUERY = "customer A SAQR"
# QUERY = "reefer container"
QUERY = "reefer"
# QUERY = "container"
# QUERY = "dry container"
# QUERY = "dry"
# QUERY = "high cubic"

# Atlas Search 집계 파이프라인
search_pipeline = [
    {
        '$search': {                        # 검색 단계
            'index': SEARCH_INDEX,          # 검색에 사용할 Search Index 명
            'compound': {                   # 복합 검색
                'should': [                 # 복합 검색 연산자 - 'should' : OR 연산 -> 각 하위 쿼리 결과의 점수 합산
                    {
                        'text': {                       # 검색 유형 지정 - 'text'==본문 검색
                            'query': QUERY,             # 검색 input
                            'path': {'wildcard': "*"},  # 검색 대상 필드 경로 - wildcard를 이용하여 모든 필드를 대상으로 지정
                            'fuzzy': {                  # 사용자 입력 오타에 대응하기 위한 fuzzy 검색 설정
                                'maxEdits': 1,          # 최대 1글자 차이 허용
                                'prefixLength': 1,      # 맨 앞에서부터 1글자까지는 정확히 일치해야 함
                                'maxExpansions': 50     # 최대 50가지의 경우의 수 허용
                            },
                            'score': { 'boost': { 'value': 10 }}    # 검색 조건과 일치하면 추가 점수
                        }
                    },
                    {
                        'text': {
                            'query': QUERY,
                            'path': "email_path",                   # 검색 대상 필드 경로 - 'email_path' 필드
                            'score': { 'boost': { 'value': 10 }}    # 검색 조건과 일치하면 추가 점수
                        }
                    }
                ]
            },
            'concurrent': True,         # 검색 병렬 실행 요청 여부
            'highlight': {                  # 검색 결과 강조 표시 설정
                'path': {'wildcard': "*"}   # 강조 표시 대상 필드 경로 - wildcard를 이용하여 모든 필드를 대상으로 지정
            }
        }
    },
    {
        '$set': {                                           # 결과 데이터 수정 단계 - metadata를 함께 출력하기 위함
            'score': { '$meta': "searchScore" },            # 검색 결과 점수 필드 추가
            'highlights': { '$meta': "searchHighlights" }   # 검색 결과 강조 표시 데이터 추가 : 검색어와 일치한 부분에 대한 정보
        }
    },
    {
        '$project': {       # projection 단계 : 검색 결과로 출력할 필드 지정
            '_id': 0        # '_id' 필드 제외, 그외 모든 필드 포함
        }
    }
]

results = bkg_collection.aggregate(pipeline=search_pipeline).to_list()

for d in results:
    # print(d)
    print(f"{d['email_path']}")
    print(f"customer ::: {d['customer name']}")
    print(f"container type ::: {d['container type']}")
    print(f"container size ::: {d['container size']}")
    print(f"score: {d['score']}")
    print(f"match:")
    for match in d['highlights']:
        match_str = " , ".join([obj['value'] for obj in match['texts'] if obj['type'] == "hit"])
        print(f"\t{match['path']} - {match_str}")
    print()

print(f"\n{len(results)} results")

./data_booking/booking_data_cherry20240715702 5.pdf
customer ::: A_SAQR_INTERNATIONAL_FREIGHT
container type ::: REEFER CONTAINER
container size ::: 45 HIGH CUBIC CONTAINER
score: 31.071537017822266
match:
	subject - Reefer
	container type - REEFER
	booking request summary - Reefer
	remark - REEFER

./data_booking_/booking_data_cherry20240715702.pdf
customer ::: A_SAQR_INTERNATIONAL_FREIGHT
container type ::: MISSING
container size ::: 45 HIGH CUBIC
score: 24.370813369750977
match:
	subject - Reefer
	booking request summary - Reefer
	remark - REEFER

./data_booking_/booking_data_cherry20240715718.pdf
customer ::: ZAYED_GLOBAL_TRANSPORT_SOLUTIONS
container type ::: REEFER
container size ::: 45 HIGH CUBIC
score: 18.880077362060547
match:
	container type - REEFER
	booking request summary - Reefer
	remark - REEFER

./data_booking/booking_data_cherry20240715703 5.pdf
customer ::: AL_WASL_GLOBAL_FREIGHT_&_CARGO
container type ::: REEFER CONTAINER
container size ::: 45 HIGH CUBIC CONTAINER
sc